In [1]:
set.seed(2000)
library(tidyverse)
library(tidymodels)
library(repr)
library(cowplot)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
cleveland <- read_csv("cleveland/processed.cleveland.data")
cleveland

New names:
• `1.0` -> `1.0...2`
• `1.0` -> `1.0...3`
• `1.0` -> `1.0...6`
• `0.0` -> `0.0...9`
• `0.0` -> `0.0...12`
Rows: 302 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): 0.0...12, 6.0
dbl (12): 63.0, 1.0...2, 1.0...3, 145.0, 233.0, 1.0...6, 2.0, 150.0, 0.0...9...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


63.0,1.0...2,1.0...3,145.0,233.0,1.0...6,2.0,150.0,0.0...9,2.3,3.0,0.0...12,6.0,0
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1
38,1,3,138,175,0,0,173,0,0.0,1,?,3.0,0


In [3]:
colnames(cleveland) <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang","oldpeak","slope","ca","thal","num")





cleveland
                         


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1
38,1,3,138,175,0,0,173,0,0.0,1,?,3.0,0


In [4]:
cleveland_mutate <- mutate(cleveland, ca = as.numeric(ca), thal = as.numeric(thal)) 
cleveland_mutate


Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”
Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,1,4,130,131,0,0,115,1,1.2,2,1,7,3
57,0,2,130,236,0,2,174,0,0.0,2,1,3,1
38,1,3,138,175,0,0,173,0,0.0,1,NA,3,0


In [5]:
cleveland_split <- initial_split(cleveland_mutate, prop = 0.80, strata = cp)
cleveland_train <- training(cleveland_split) 
cleveland_test <- testing(cleveland_split)

In [6]:
cleveland_train_scaled <- cleveland_train |> 
 mutate(age = scale(age, center = TRUE), 
        sex = scale(sex, center = TRUE), 
        cp = scale(cp, center = TRUE), 
        trestbps = scale(trestbps, center = TRUE), 
        chol = scale(chol, center = TRUE), 
        fbs = scale(fbs, center = TRUE), 
        restecg = scale(restecg, center = TRUE), 
        thalach = scale(thalach, center = TRUE),
        exang = scale(exang, center = TRUE),
        oldpeak = scale(oldpeak, center = TRUE),
        slope = scale(slope, center = TRUE),
        ca = scale(ca, center = TRUE),
        thal = scale(thal, center = TRUE),
        num = scale(num, center = TRUE))
        
cleveland_train_scaled

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>"
-1.4820032,-1.4659139,-1.231003,-0.1525805,-0.8226711,-0.4328221,1.0404684,0.9599115,-0.6727938,0.2815177,-0.9958610,-0.7230651,-0.8628835,-0.7500609
0.1790216,0.6793259,-1.231003,-0.7159546,-0.1666526,-0.4328221,-0.9733414,1.2206193,-0.6727938,-0.2134585,-0.9958610,-0.7230651,-0.8628835,-0.7500609
0.1790216,-1.4659139,-1.231003,0.4107936,1.0223809,-0.4328221,1.0404684,0.1343369,-0.6727938,0.1990217,0.6411708,-0.7230651,-0.8628835,-0.7500609
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
0.5112265,0.6793259,0.856917,1.7628915,-1.39668734,2.3007912,1.0404684,-2.6030946,-0.6727938,-0.04846642,0.6411708,1.3707185,0.6877116,1.61855237
0.2897566,-1.4659139,0.856917,0.4107936,-0.06414973,-0.4328221,-0.9733414,-1.1692019,1.4801464,-0.70843467,0.6411708,-0.7230651,1.2045767,0.03947689
1.5078414,0.6793259,0.856917,0.6361433,-1.04817750,2.3007912,-0.9733414,-0.3870786,-0.6727938,1.93143834,0.6411708,1.3707185,1.2045767,0.82901463


In [7]:

cleveland <- filter(cleveland_train, age >= 60) 

cleveland









age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
64,1,1,110,211,0,2,144,1,1.8,2,0,3,0
69,0,1,140,239,0,0,151,0,1.8,1,2,3,0
65,1,1,138,282,1,2,174,0,1.4,2,1,3,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
61,1,4,148,203,0,0,161,0,0.0,1,1,7,2
63,1,4,140,187,0,2,144,1,4.0,1,2,7,2
68,1,4,144,193,1,0,141,0,3.4,2,2,7,2


In [8]:
trestbps_mean <- summarize(cleveland_train, trestbps_mean = mean(trestbps))
trestbps_mean

trestbps_mean
<dbl>
132.7083
